In [7]:
!pip install pytorch-lightning --quiet
!pip install transformers --quiet
!pip install datasets --quiet

Model and Data Sources por Cañete, J., Chaperon, G., Fuentes, R., Pérez, J., & Bustos, B. (2020). Spanish Pre-Trained BERT Model and Evaluation Data. Recuperado de https://arxiv.org/abs/2308.02976

# Spanish Pre-Trained BERT Model and Evaluation Data

@inproceedings{CaneteCFP2020,
  title={Spanish Pre-Trained BERT Model and Evaluation Data},
  author={Cañete, José and Chaperon, Gabriel and Fuentes, Rodrigo and Ho, Jou-Hui and Kang, Hojin and Pérez, Jorge},
  booktitle={PML4DC at ICLR 2020},
  year={2020}
}

In [42]:
import pickle
import numpy as np
import pandas as pd
import torch
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
from transformers import BertModel, AutoTokenizer, AdamW
import datasets
import torch.nn.functional as F
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
# %load_ext tensorboard
from tensorboard.plugins.hparams import api as hp
import tensorflow as tf
import torch
from sklearn.model_selection import train_test_split
import time

# Hyperparámetros del modelo básicos: modelo preentrenado de BERT en español y longitud máxima de secuencia
BERT_MODEL_NAME="dccuchile/bert-base-spanish-wwm-cased"
MAX_LEN = 128

class DataModule(pl.LightningDataModule):
    def __init__(self, train_path, val_path, test_path, batch_size, tokenizer, max_length):
        super().__init__()
        self.train_path = train_path
        self.val_path = val_path
        self.test_path = test_path
        self.tokenizer = AutoTokenizer.from_pretrained(BERT_MODEL_NAME)
        self.batch_size = batch_size
        self.max_length = max_length
        self.tokenizer = tokenizer
        self.stop_words = set(stopwords.words('spanish'))  # Asumiendo que los textos están en español

    def preprocess_text(self, text):
        '''
        la funcion preprocess_text recibe un texto y realiza las siguientes operaciones:
        - Tokeniza el texto
        - Elimina stopwords
        - Une los tokens filtrados en un solo texto
        parámetros:
        - text: texto a preprocesar
        return:
        - preprocessed_text: texto preprocesado
        '''
        # Tokenización
        tokens = word_tokenize(text)
        # Eliminación de stopwords
        filtered_tokens = [token for token in tokens if token not in self.stop_words]
        # Unir los tokens filtrados en un solo texto
        preprocessed_text = ' '.join(filtered_tokens)
        
        return preprocessed_text

    def load_dataset(self, path):
        '''
        la función load_dataset recibe la ruta de un archivo de texto y carga los datos en un DataFrame de pandas.
        Luego, aplica el preprocesamiento de datos y aplica la funcion encode a la columna 'text' del DataFrame. Finalmente,
        convierte los labels a tensores y depués el DataFrame en un objeto de tipo Dataset de la librería datasets compatible con torch
        parámetros:
        - path: ruta del archivo de texto
        return:
        - dataset: objeto de tipo Dataset de la librería datasets compatible con torch
        '''
        df = pd.read_csv(path, sep='\t', header=None, names=['cat', 'text'])
        df['labels'] = df.cat.map({0: 0, 1: 1})
        df = df[['text', 'labels']]
        df['labels'] = df['labels'].astype(np.int64)
        df['text'] = df['text'].apply(self.preprocess_text)
        df['labels'] = torch.tensor(df['labels'].values)
        dataset = datasets.Dataset.from_pandas(df)
        dataset = dataset.map(lambda examples: self.encode(examples), batched=True)
        dataset.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'labels'])
        
        return dataset

    def encode(self, examples):
        '''
        la función encode recibe una columna de texto y aplica la tokenización de BERT a cada texto aplicando una serie de transformaciones y parámetros para optimizar la carga de datos
        parámetros:
        - examples: columna de texto a tokenizar
        return:
        - tokenized_text: texto tokenizado
        '''
        return self.tokenizer(examples['text'], add_special_tokens=True, truncation=True, padding='max_length', max_length=MAX_LEN, return_attention_mask=True, return_tensors='pt' )

    def train_dataloader(self):
        '''
        la función train_dataloader carga los datos de entrenamiento y los convierte en un DataLoader de torch
        return:
        - DataLoader de torch con los datos de entrenamiento        
        '''
        dataset = self.load_dataset(self.train_path)
        return torch.utils.data.DataLoader(dataset,
                                           batch_size=self.batch_size,
                                           shuffle=True,
                                           num_workers=4,
                                           persistent_workers=True)

    def val_dataloader(self):
        '''
        la función val_dataloader carga los datos de validación y los convierte en un DataLoader de torch
        return:
        - DataLoader de torch con los datos de validación
        '''
        dataset = self.load_dataset(self.val_path)
        return torch.utils.data.DataLoader(dataset,
                                           batch_size=self.batch_size,
                                           shuffle=False,
                                           num_workers=4,
                                           persistent_workers=True)

    def test_dataloader(self):
        ''''
        la funcion test_dataloader carga los datos de test y los convierte en un DataLoader de torch
        return:
        - DataLoader de torch con los datos de test
        '''
        dataset = self.load_dataset(self.test_path)
        return torch.utils.data.DataLoader(dataset,
                                           batch_size=self.batch_size,
                                           shuffle=False,
                                           num_workers=4,
                                           persistent_workers=True)

In [43]:
HP_LEARNING_RATE = hp.HParam('learning_rate', hp.RealInterval(1e-5, 2e-5))
HP_DROPOUT = hp.HParam('dropout_prob', hp.RealInterval(0.3, 0.5))
HP_BATCH_SIZE = hp.HParam('batch_size', hp.Discrete([16, 32]))
METRIC_ACCURACY = 'accuracy'
hparams = {
    HP_LEARNING_RATE: 2e-5,
    HP_DROPOUT: 0.3,
    HP_BATCH_SIZE: 16,
}

with tf.summary.create_file_writer('C:\\TFG\\Hyperparameters').as_default():
    hp.hparams_config(
        hparams=[HP_LEARNING_RATE, HP_DROPOUT, HP_BATCH_SIZE],
        metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')],
    )

In [44]:
class BertSentimentClassifier(pl.LightningModule):
    def __init__(self):
        super(BertSentimentClassifier, self).__init__()
        self.save_hyperparameters(hparams)
        self.learning_rate = torch.tensor(self.hparams[HP_LEARNING_RATE])
        self.batch_size = torch.tensor(self.hparams[HP_BATCH_SIZE])
        self._frozen = False
        # loss function
        self.criterion = torch.nn.CrossEntropyLoss()
        # bert model
        self.bert = BertModel.from_pretrained(BERT_MODEL_NAME)
        self.dropout = torch.nn.Dropout(self.hparams[HP_DROPOUT])
        self.fc = torch.nn.Linear(self.bert.config.hidden_size, 2)

    def configure_optimizers(self):
        '''
        la función configure_optimizers configura el optimizador AdamW con los hiperparámetros definidos en hparams para el modelo BERT
        return:
        - optimizer: optimizador AdamW
        '''
        optimizer = AdamW(self.parameters(), lr=self.hparams[HP_LEARNING_RATE])

        return optimizer

    def forward(self, batch):
        '''
        la función forward recibe un batch de datos y realiza el forward pass del modelo BERT, para esto aplica la tokenización de BERT a los datos de entrada y obtiene los logits y las probabilidades de las predicciones
        parámetros:
        - batch: batch de datos
        return:
        - logits: logits de las predicciones
        '''
        b_input_ids = batch['input_ids']
        b_input_mask = batch['attention_mask']
        b_token_type_ids = batch['token_type_ids']

        outputs = self.bert(input_ids=b_input_ids,
                            attention_mask=b_input_mask,
                            token_type_ids= b_token_type_ids,
                            return_dict=True)
        pooled_output = outputs.pooler_output
        pooled_output = self.dropout(pooled_output)
        logits = self.fc(pooled_output)
        # Aplicar softmax para obtener probabilidades
        probabilities = F.softmax(logits, dim=1)
        return logits, probabilities

    def on_train_start(self):
        '''
        la función on_train_start se ejecuta al inicio del entrenamiento y guarda los hiperparámetros en TensorBoard
        '''
        self.logger.log_hyperparams(self.hparams, {'hp/metric': 0})
    
    def training_step(self, batch, batch_idx):
        '''
        la función training_step recibe un batch de datos y realiza el forward pass del modelo BERT, calcula la loss y la precisión de las predicciones, además guarda los valores de loss y accuracy en TensorBoard
        parámetros:
        - batch: batch de datos
        - batch_idx: índice del batch (se utiliza para implementar técnicas de entrenamiento como gradient accumulation)
        return:
        - loss: loss de las predicciones
        '''
        logits, probabilities = self.forward(batch)
        loss = self.criterion(logits, batch['labels'])
        # Calcular la precisión usando probabilidades
        predictions = torch.argmax(probabilities, dim=1)
        accuracy = (batch['labels'] == predictions).float().mean()
        self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        self.log('train_accuracy', accuracy, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        
        return {'loss': loss, 'accuracy': accuracy}
        

    def validation_step(self, batch, batch_idx):
        '''
        la función validation_step recibe un batch de datos y realiza el forward pass del modelo BERT, calcula la loss y la precisión de las predicciones, además guarda los valores de loss y accuracy en TensorBoard
        parámetros:
        - batch: batch de datos 
        - batch_idx: índice del batch
        return:
        - loss: loss de las predicciones
        '''
        logits, probabilities = self.forward(batch)
        loss = self.criterion(logits, batch['labels'])
        # Calcular la precisión usando probabilidades
        predictions = torch.argmax(probabilities, dim=1)
        accuracy = (batch['labels'] == predictions).float().mean()
                
        # Log the validation loss and accuracy for visualization in TensorBoard
        self.log('val_loss', loss, on_epoch=True, prog_bar=True)
        self.log('val_accuracy', accuracy, on_epoch=True, prog_bar=True)
        
        # Return the metrics
        return {'val_loss': loss, 'val_accuracy': accuracy}

    def test_step(self, batch, batch_idx):
        '''
        la función test_step recibe un batch de datos y realiza el forward pass del modelo BERT, calcula la loss y la precisión de las predicciones, además guarda los valores de loss y accuracy en TensorBoard
        parámetros:
        - batch: batch de datos
        - batch_idx: índice del batch
        return:
        - loss: loss de las predicciones
        '''
        logits, probabilities = self.forward(batch)
        loss = self.criterion(logits, batch['labels'])
        # Calcular la precisión usando probabilidades
        predictions = torch.argmax(probabilities, dim=1)
        accuracy = (batch['labels'] == predictions).float().mean()
        
        # Log the test loss and accuracy for visualization in TensorBoard
        self.log('test_loss', loss, on_epoch=True, prog_bar=True)
        self.log('test_accuracy', accuracy, on_epoch=True, prog_bar=True)
        
        # Return the metrics
        return {'test_loss': loss, 'test_accuracy': accuracy}


In [45]:
#cargar datos
train_path = 'C:\TFG\RawData\Catalonian independence corpus\spanish_train.csv'
val_path = 'C:\TFG\RawData\Catalonian independence corpus\spanish_val.csv'
test_path = 'C:\TFG\RawData\Catalonian independence corpus\spanish_test.csv'

In [46]:
# se transforman los datos en DataFrames de pandas
train = pd.read_csv(train_path, delimiter='\t', encoding='utf-8', on_bad_lines='skip')
val = pd.read_csv(val_path, delimiter='\t', encoding='utf-8', on_bad_lines='skip')
test = pd.read_csv(test_path, delimiter='\t', encoding='utf-8', on_bad_lines='skip')

In [47]:
# concatenar
df = pd.concat([train, val, test], ignore_index=True)


In [48]:
# se observan las columnas
df.columns

Index(['id_str', 'TWEET', 'LABEL'], dtype='object')

In [49]:
# frecuencia de LABEL
df['LABEL'].value_counts()

LABEL
AGAINST    4105
FAVOR      4104
NEUTRAL    1868
Name: count, dtype: int64

In [50]:
# drop id_str columna
df = df.drop(columns=['id_str'])

In [51]:
# rename LABEL a cat y TWEET a text
df = df.rename(columns={'LABEL': 'cat', 'TWEET': 'text'})


In [52]:
def label_to_labels(label):
    '''
    la funcion label_to_labels recibe una etiqueta y la convierte en un valor numérico según la siguiente relación:
    parámetros:
    - label: etiqueta a convertir
    return:
    - valor numérico de la etiqueta
    '''
    if label == 'NEUTRAL':
        return 2
    elif label == 'FAVOR':
        return 1
    else:
        return 0

df['cat'] = df['cat'].apply(label_to_labels)

In [53]:
# se filtran las columnas del dataset
df = df[['cat', 'text']]

In [54]:
# se observan las frecuencias por categorías
df['cat'].value_counts()

cat
0    4105
1    4104
2    1868
Name: count, dtype: int64

Se prescinde de la etiqueta "Neutral" dado que hemos comprobado que en su gran mayoría contiene textos que no están escritos en español. Sino en portugués, gallego, etc. Por lo tanto, se ha decidido trabajar con las etiquetas "Positive" y "Negative". Ya que puede perjudicar a la precisión del modelo.

In [55]:
# 1868 instancias aleatorias de clase Negativo
independencia_df_es_negativo = df[df['cat'] == 0].sample(4104)
# 1868 instancias aleatorias de clase Positivo
independencia_df_es_positivo = df[df['cat'] == 1].sample(4104)
# 1868 instancias aleatorias de clase Neutral
independencia_df_es_neutral = df[df['cat'] == 2].sample(1868)
# concatenar los subconjuntos
independencia_df_es_balanced = pd.concat([independencia_df_es_negativo, independencia_df_es_positivo])
independencia_df_es_balanced = independencia_df_es_balanced.sample(frac=1)
# se muestra el resultado
independencia_df_es_balanced['cat'].value_counts()

cat
0    4104
1    4104
Name: count, dtype: int64

In [56]:
# dividir en train, val y test
train, test = train_test_split(independencia_df_es_balanced, test_size=0.2, random_state=1335)
train, val = train_test_split(train, test_size=0.25, random_state=1335)
# guardar en .txt
train.to_csv('C:\TFG\DataProcessed\independencia_train.txt', sep='\t', index=False, header=False)
val.to_csv('C:\TFG\DataProcessed\independencia_val.txt', sep='\t', index=False, header=False)
test.to_csv('C:\TFG\DataProcessed\independencia_test.txt', sep='\t', index=False, header=False)

In [57]:
print("Is CUDA available:", torch.cuda.is_available())
print("CUDA version:", torch.version.cuda)
print("cuDNN version:", torch.backends.cudnn.version())

Is CUDA available: True
CUDA version: 12.1
cuDNN version: 8801


In [58]:
# Hyper-parameters
BATCH_SIZE = 16 
NUM_EPOCHS = 2
tokenizer = AutoTokenizer.from_pretrained(BERT_MODEL_NAME)
torch.manual_seed(1335)
torch.set_float32_matmul_precision('medium')
data = DataModule('C:\TFG\DataProcessed\independencia_train.txt', 'C:\TFG\DataProcessed\independencia_val.txt', 'C:\TFG\DataProcessed\independencia_test.txt', BATCH_SIZE, tokenizer, MAX_LEN)
logdir = "C:\TFG\Models\\bert_logs"
logger = TensorBoardLogger(logdir, name="bert")
model = BertSentimentClassifier()


c:\Users\Lenovo\anaconda3\envs\ChromaDB\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of BertModel were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [59]:
print(model)

BertSentimentClassifier(
  (criterion): CrossEntropyLoss()
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(31002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): 

In [60]:
# Entrenar el modelo y medir el tiempo de ejecución
start_time = time.time()
beto_trainer = pl.Trainer(max_epochs = NUM_EPOCHS, logger=logger, accelerator="gpu")
beto_trainer.fit(model, datamodule=data)
finish_time = time.time()
beto_time = finish_time - start_time


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
c:\Users\Lenovo\anaconda3\envs\ChromaDB\lib\site-packages\transformers\optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(

  | Name      | Type             | Params
-----------------------------------------------
0 | criterion | CrossEntropyLoss | 0     
1 | bert      | BertModel        | 109 M 
2 | dropout   | Dropout          | 0     
3 | fc        | Linear           | 1.5 K 
-----------------------------------------------
109 M     Trainable params
0         Non-trainable params
109 M     Total params
439.410   Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]


Map: 100%|██████████| 1642/1642 [00:01<00:00, 1226.88 examples/s]



Map: 100%|██████████| 4924/4924 [00:01<00:00, 2504.87 examples/s]


Epoch 0: 100%|██████████| 308/308 [00:52<00:00,  5.82it/s, v_num=46, train_loss_step=0.460, train_accuracy_step=0.750]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 1: 100%|██████████| 308/308 [00:41<00:00,  7.36it/s, v_num=46, train_loss_step=0.724, train_accuracy_step=0.667, val_loss=0.535, val_accuracy=0.719, train_loss_epoch=0.626, train_accuracy_epoch=0.639] 
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 1: 100%|██████████| 308/308 [00:46<00:00,  6.69it/s, v_num=46, train_loss_step=0.724, train_accuracy_step=0.667, val_loss=0.505, val_accuracy=0.739, train_loss_epoch=0.417, train_accuracy_epoch=0.816]

`Trainer.fit` stopped: `max_epochs=2` reached.


Epoch 1: 100%|██████████| 308/308 [00:55<00:00,  5.54it/s, v_num=46, train_loss_step=0.724, train_accuracy_step=0.667, val_loss=0.505, val_accuracy=0.739, train_loss_epoch=0.417, train_accuracy_epoch=0.816]


In [61]:
# Probar el modelo con el conjunto de test
test_out = beto_trainer.test(model, datamodule=data)
print(test_out)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Epoch 0:  15%|█▍        | 45/308 [10:49<1:03:14,  0.07it/s, v_num=45, train_loss_step=0.763, train_accuracy_step=0.500]



Map: 100%|██████████| 1642/1642 [00:00<00:00, 2725.67 examples/s]


Testing DataLoader 0: 100%|██████████| 103/103 [00:08<00:00, 12.12it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.7423873543739319     │
│         test_loss         │    0.5025784969329834     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.5025784969329834, 'test_accuracy': 0.7423873543739319}]


In [62]:
# Guardar el data.tokenizer
data.tokenizer.save_pretrained('C:\\TFG\\Models\\bert\\independencia_tokenizer')
# cargar el data.tokenizer
tokenizer = AutoTokenizer.from_pretrained('C:\\TFG\\Models\\bert\\independencia_tokenizer')


('C:\\TFG\\Models\\bert\\independencia_tokenizer\\tokenizer_config.json',
 'C:\\TFG\\Models\\bert\\independencia_tokenizer\\special_tokens_map.json',
 'C:\\TFG\\Models\\bert\\independencia_tokenizer\\vocab.txt',
 'C:\\TFG\\Models\\bert\\independencia_tokenizer\\added_tokens.json',
 'C:\\TFG\\Models\\bert\\independencia_tokenizer\\tokenizer.json')

In [63]:
# predecir la clase de un tweet con trainer.predict
tweet = "El 22/02 doy una conferencia de física cuántica:  Las escisiones del Uranio con isótopos en las tres fases ciclotrónicas de este elemento   Confieso que de física cuántica no tengo ni puta idea, pero viendo a @pablocasado_ hablar de democracia y DDHH,  me he venido arriba! ???"
tweet = data.tokenizer(tweet, add_special_tokens=True, truncation=True, padding='max_length', max_length=MAX_LEN, return_attention_mask=True, return_tensors='pt')
tweet = {k: v.to(model.device) for k, v in tweet.items()}
model.eval()
output = model(tweet)
predicted_class = torch.argmax(output[0], dim=1)
print(predicted_class)


tensor([1])


In [65]:
# guardar el modelo con pickle
with open('C:\\TFG\\Models\\bert\\beto_model.pickle', 'wb') as f:
    pickle.dump(model, f)

In [66]:
# guardar el modelo con joblib
import joblib

joblib.dump(model, 'C:\\TFG\\Models\\bert\\beto_model.joblib')


['C:\\TFG\\Models\\bert\\beto_model.joblib']

In [67]:
# guardar el modelo con torch
torch.save(model, 'C:\\TFG\\Models\\bert\\beto_model.pth')


In [74]:
# cargar el modelo
model_path = 'C:\\TFG\\Models\\bert\\beto_model.pth'
model = BertSentimentClassifier()
model.load_state_dict(torch.load(model_path))
tweet = "El 22/02 doy una conferencia de física cuántica:  Las escisiones del Uranio con isótopos en las tres fases ciclotrónicas de este elemento   Confieso que de física cuántica no tengo ni puta idea, pero viendo a @pablocasado_ hablar de democracia y DDHH,  me he venido arriba! ???"
tweet = data.tokenizer(tweet, add_special_tokens=True, truncation=True, padding='max_length', max_length=MAX_LEN, return_attention_mask=True, return_tensors='pt', return_token_type_ids=True)
tweet = {k: v.to(model.device) for k, v in tweet.items()}
model.eval()
output = model(tweet)
predicted_class = torch.argmax(output[0], dim=1)
print(predicted_class)

Some weights of BertModel were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tensor([1])


In [79]:
# guardar beto_time como dataset
beto_time = pd.DataFrame([beto_time])


,0
0,256.17769


In [80]:
# cambiar nombre de columna a beto_time
beto_time.columns = ['beto_time']

In [82]:
# guardar beto_time
beto_time.to_csv('C:\\TFG\\Metrics\\beto_time.csv', index=False)